<a href="https://colab.research.google.com/github/dxspimentel/PUC/blob/main/MVP_Diego_Pimentel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MVP Análise de Dados e Boas Práticas**

Nome: Diego Pimentel <br>
Matrícula: 4052025000258 <br>
Dataset: **Escolas Conectadas**

<br>
--- OBSERVAÇÃO IMPORTANTE ---

Este é um Dataset adaptado, devido a Segurança da Informação e Sigilo da Instituição.

------

# Descrição do Problema

Dados relacionados ao projeto *Escolas Conectadas*, cujo objetivo é levar internet a escolas com estrutura precária, ou sem estrutura, nas regiões mais afastadas das grandes cidades.<br>
Neste trabalho será desenvolvida uma solução que facilite a identificação de quais escolas foram de fato entregues, sem pendências, dentro do universo de escolas com obras já contratadas.

## Hipóteses do Problema

Cada escola tem a sua particularidade, com isso trabalha com escopo específico.
O escopo de cada projeto é dividido em 3 tipos de obras:<br>
1) Rede Interna<br>
2) Rede Externa<br>
3) Gerador Solar<br>

A questão é analisar se as etapas previstas em cada escola foram concluídas e com isso classificar como **Conectada**.

## Tipo de Problema

Este é um problema de **classificação supervisionada**.<br>
Cada etapa do projeto é classificada de acordo com o status de execução de obra. Uma vez que o escopo esteja totalmente atendido, a escola é classificada como entregue (**Conectada**).

## Seleção de Dados

Para este trabalho considerei duas bases de dados. A primeira com o monitoramento das obras, por projeto, e a segunda informando o escopo a ser considerado para cada escola.<br><br>

A base de projetos está verticalizada, ou seja, os status das etapas estão empilhados. Já a base de escopo já está bem definida, trazendo os escopos em 3 colunas distintas, para cada projeto.

# Importação das Bibliotecas Necessárias e Carga de Dados

Esta seção consolida todas as importações de bibliotecas necessárias para a análise, visualização e pré-processamento dos dados, bem como o carregamento inicial das bases para montar o dataset **Escolas Conectadas**.

In [23]:
# Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# URLs
github_projetos = 'https://raw.githubusercontent.com/dxspimentel/PUC/main/PUC_Db_01_Base_Projetos.csv'
github_escopo = 'https://raw.githubusercontent.com/dxspimentel/PUC/main/PUC_Db_02_Base_Escopo.csv'

In [24]:
# Carregamento da Base de Projetos
df_projetos = pd.read_csv(github_projetos, encoding='latin-1')
print('df_projetos carregado com sucesso')

df_projetos carregado com sucesso


In [25]:
# Carregamento da Base de Escopos
df_escopo = pd.read_csv(github_escopo, encoding='latin-1')
print('df_escopo carregado com sucesso')

df_escopo carregado com sucesso
